# Part 1: Graph Database Loading

## Outline
1. Explore Data
2. Review Target Graph Schema
3. Configure Neo4j Connection
4. Load Nodes
5. Load Relationships

In [2]:
%%capture
%pip install python-dotenv graphdatascience

In [3]:
from typing import Tuple
import pandas as pd
import numpy as np
from graphdatascience import GraphDataScience
from dotenv import load_dotenv
import os

## Explore & Load Data
The dataset we are going to use is healthcare expense claims with anonymised beneficiaries, claims and providers.

We will predict the potentially fraudulent providers based on the claims filed by them. We will use a GDS embedding algorithm to chart out Fraudulent patterns in the provider's claims to understand the future behaviour of providers.

Before loading data into any Database, we usually have to come up with a schema and implement it on the Database. Graph Data Modelling is an important step with Neo4j and you define it based on the questions you would like to ask on the graph.

If you are interested, you can explore the data in-depth.

In [36]:
provider_feat_df = pd.read_csv('gs://neo4j-datasets/insurance-claim/baseline.csv')
provider_feat_df['providerId'] = provider_feat_df.provider.str.slice(start=3).astype(int)
provider_feat_df.to_csv('baseline.csv', index=False)
provider_feat_df

,provider,potentialFraudInd,renalDiseaseIndicatorEnc,chronicCondAlzheimerEnc,chronicCondHeartfailureEnc,chronicCondKidneyDiseaseEnc,chronicCondCancerEnc,chronicCondObstrPulmonaryEnc,chronicCondDepressionEnc,chronicCondDiabetesEnc,chronicCondIschemicHeartEnc,chronicCondOsteoporasisEnc,chronicCondrheumatoidarthritisEnc,chronicCondstrokeEnc,claimCount,providerId
0,PRV51001,0,0.054422,0.102041,0.129252,0.115646,0.034014,0.068027,0.061224,0.142857,0.156463,0.040816,0.054422,0.040816,25,51001
1,PRV51003,1,0.046105,0.089030,0.127186,0.101749,0.015898,0.065183,0.085851,0.158983,0.178060,0.052464,0.060413,0.019078,132,51003
2,PRV51004,0,0.034328,0.095522,0.131343,0.074627,0.023881,0.061194,0.094030,0.156716,0.161194,0.073134,0.068657,0.025373,149,51004
3,PRV51005,1,0.048776,0.080226,0.128060,0.095480,0.031073,0.055556,0.091337,0.150471,0.168550,0.064783,0.062335,0.023352,1165,51005
4,PRV51007,0,0.035370,0.083601,0.128617,0.070740,0.038585,0.051447,0.093248,0.157556,0.163987,0.067524,0.070740,0.038585,72,51007
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5405,PRV57759,0,0.032895,0.092105,0.131579,0.098684,0.026316,0.072368,0.059211,0.131579,0.184211,0.085526,0.059211,0.026316,28,57759
5406,PRV57760,0,0.000000,0.038961,0.142857,0.025974,0.000000,0.025974,0.090909,0.233766,0.285714,0.142857,0.012987,0.000000,22,57760
5407,PRV57761,0,0.054762,0.085714,0.133333,0.095238,0.033333,0.071429,0.090476,0.130952,0.147619,0.076190,0.057143,0.023810,82,57761
5408,PRV57762,0,0.000000,0.000000,0.000000,0.200000,0.200000,0.200000,0.000000,0.200000,0.200000,0.000000,0.000000,0.000000,1,57762


In [37]:
claim_df = pd.read_csv('gs://neo4j-datasets/insurance-claim/claim.csv', low_memory=False)
claim_df['providerId'] = claim_df.provider.str.slice(start=3).astype(int)
claim_df

,beneID,claimID,claimStartDt,claimEndDt,provider,inscClaimAmtReimbursed,attendingPhysician,operatingPhysician,otherPhysician,admissionDt,...,chronicCondstroke,iPAnnualReimbursementAmt,iPAnnualDeductibleAmt,oPAnnualReimbursementAmt,oPAnnualDeductibleAmt,dobYear,isDeceased,maxDate,approxAge,providerId
0,BENE11001,CLM46614,2009-04-12,2009-04-18,PRV55912,26000,PHY390922,NaN,NaN,2009-04-12,...,1,36000,3204,60,70,1943,0,2009-12-01,67.0,55912
1,BENE11001,CLM66048,2009-08-31,2009-09-02,PRV55907,5000,PHY318495,PHY318495,NaN,2009-08-31,...,1,36000,3204,60,70,1943,0,2009-12-01,67.0,55907
2,BENE11001,CLM68358,2009-09-17,2009-09-20,PRV56046,5000,PHY372395,NaN,PHY324689,2009-09-17,...,1,36000,3204,60,70,1943,0,2009-12-01,67.0,56046
3,BENE11011,CLM38412,2009-02-14,2009-02-22,PRV52405,5000,PHY369659,PHY392961,PHY349768,2009-02-14,...,1,5000,1068,250,320,1914,0,2009-12-01,95.8,52405
4,BENE11011,CLM144521,2009-01-18,2009-01-18,PRV52314,50,PHY379398,NaN,NaN,NaN,...,1,5000,1068,250,320,1914,0,2009-12-01,95.8,52314
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
558206,BENE159198,CLM510792,2009-08-06,2009-08-06,PRV53699,800,PHY364188,PHY364188,PHY385752,NaN,...,0,0,0,5470,1870,1952,0,2009-12-01,57.7,53699
558207,BENE159198,CLM551294,2009-08-29,2009-08-29,PRV53702,400,PHY423019,PHY332284,NaN,NaN,...,0,0,0,5470,1870,1952,0,2009-12-01,57.7,53702
558208,BENE159198,CLM596444,2009-09-24,2009-09-24,PRV53676,60,PHY361063,NaN,NaN,NaN,...,0,0,0,5470,1870,1952,0,2009-12-01,57.7,53676
558209,BENE159198,CLM636992,2009-10-18,2009-10-18,PRV53689,70,PHY403198,NaN,PHY419379,NaN,...,0,0,0,5470,1870,1952,0,2009-12-01,57.7,53689


## Review Target Graph Schema
Given the number of entities and categorical features in this dataset, we have a lot of options for a graph schema and can create a very rich and complex data model.  However, for this tutorial, we will keep things really simple, especially as we are just getting started with this dataset.  

We will model providers and nodes and connect them directly to beneficiaries, diagnostis codes, and diagnosis group codes which also be modeled as nodes.  Each relationship represents one or more claims where the provider provides services to a beneficiary or used a code. Below is an image of the data model.

<img src="https://storage.googleapis.com/neo4j-datasets/insurance-claim/img/schema.png" width="480" >

While this simple data model does not include every entity from the dataset it will provide a good foundation for understanding the data in the graph and generating graph features.  We can always expand on the dataset more later once we have a better grasp of the graph. 

## Configure Neo4j Connection

**⚠️** You need to duplicate `config.env.template` file in the left and rename as `config.env`. Edit the values in this file and provide the values for `NEO4J_URI`, `NEO4J_PASSWORD`, `GCLOUD_REGION` (if you selected a different region) from the previous steps in this lab. 

In [ ]:
load_dotenv('config.env', override=True)
# Use Neo4j URI and credentials according to our setup
gds = GraphDataScience(
    os.getenv('NEO4J_URI'),
    auth=(os.getenv('NEO4J_USERNAME'),
          os.getenv('NEO4J_PASSWORD')),
    aura_ds=eval(os.getenv('AURA_DS').title()))

gds.set_database("neo4j")

## Load Nodes

Always start by creating uniqueness or node keys constraints. This ensures that nodes are indexed with a unique id property which is needed for loading to perform well.  
If Neo4j node or relationship loading seems to be "taking forever", double check that created and are using uniqueness or node key constriants.

In [26]:
gds.run_cypher('CREATE CONSTRAINT unique_provider_id IF NOT EXISTS FOR (n:Provider) REQUIRE n.providerId IS UNIQUE')
gds.run_cypher('CREATE CONSTRAINT unique_beneficiary_id IF NOT EXISTS FOR (n:Beneficiary) REQUIRE n.beneId IS UNIQUE')
gds.run_cypher('CREATE CONSTRAINT unique_diagnosis_code IF NOT EXISTS FOR (n:DiagnosisCode) REQUIRE n.code IS UNIQUE')
gds.run_cypher('CREATE CONSTRAINT unique_diagnosis_group_code IF NOT EXISTS FOR (n:DiagnosisGroupCode) REQUIRE n.code IS UNIQUE')

""


### Load all Provider nodes

In [27]:
print('======  loading provider nodes  ======')
node_records = provider_feat_df[['providerId', 'potentialFraudInd']].to_dict('records')

res = gds.run_cypher('''
    UNWIND $data AS d
    MERGE(n:Provider {providerId: d.providerId})
    SET n.potentialFraudInd = d.potentialFraudInd
    RETURN count(n) AS nodeLoadedCount
''', params={'data': node_records})
print(f'Complete: Loaded {res.nodeLoadedCount[0]:,} nodes.')


======  loading provider nodes  ======
Complete: Loaded 5,410 nodes.


### Load all Beneficiary nodes

In [28]:
print('======  loading beneficiary nodes  ======')
res = gds.run_cypher('''
    UNWIND $nodeKeys AS nodeKey
    MERGE(n:Beneficiary {beneId: nodeKey})
    RETURN count(n) AS nodeLoadedCount
''', params={'nodeKeys': claim_df.beneID.drop_duplicates().tolist()})
print(f'Complete: Loaded {res.nodeLoadedCount[0]:,} nodes.')

======  loading beneficiary nodes  ======
Complete: Loaded 138,556 nodes.


### Load all Diagnosis Claim Codes as nodes

In [29]:
# Diagnosis Claim Code Nodes
diagnosis_code_list = []
for col in claim_df.columns:
    if 'diagnosiscode' in col.lower():
        diagnosis_code_list.extend(claim_df[col].dropna().drop_duplicates().tolist())

print('======  loading diagnosis code nodes  ======')
res = gds.run_cypher('''
    UNWIND $nodeKeys AS nodeKey
    MERGE(n:DiagnosisCode {code: nodeKey})
    RETURN count(n) AS nodeLoadedCount
''', params={'nodeKeys': pd.Series(diagnosis_code_list).drop_duplicates().tolist()})
print(f'Complete: Loaded {res.nodeLoadedCount[0]:,} nodes') 

======  loading diagnosis code nodes  ======
Complete: Loaded 11,040 nodes


### Load all Diagnosis Group Codes as nodes

In [30]:
print('======  loading diagnosis group code nodes  ======')
res = gds.run_cypher('''
    UNWIND $nodeKeys AS nodeKey
    MERGE(n:DiagnosisGroupCode {code: nodeKey})
    RETURN count(n) AS nodeLoadedCount
''', params={'nodeKeys': claim_df['diagnosisGroupCode'].dropna().drop_duplicates().tolist()})
print(f'Complete: Loaded {res.nodeLoadedCount[0]:,} nodes')

======  loading diagnosis group code nodes  ======
Complete: Loaded 736 nodes


## Load Relationships

There are going to millions of relationships in this dataset.  While not strictly necessary lets create a function that will allow us to load the relationships in chunks. This will provide better visibility into the loading and can help make it more effcicient.  For larger datasets it will also protect against out-of-memory issues. 

In [31]:
def chunks(xs, n=50_000):
    n = max(1, n)
    return [xs[i:i + n] for i in range(0, len(xs), n)]

In [32]:
# Beneficiary relationship loading will will perform best if we aggregate to unique pairs prior to sending the query
print(f'======  loading HAS_CLAIMS_WITH_BENE relationships  ======')
tmp_df = claim_df[['providerId', 'beneID', 'claimID', 'inscClaimAmtReimbursed', 'inpatient']] \
    .groupby(['providerId', 'beneID']).agg({
        'claimID': 'count',
        'inscClaimAmtReimbursed': sum,
        'inpatient': sum
    }).reset_index()
rel_records = tmp_df.to_dict('records')
total = len(rel_records)
print(f'staged {total:,} records...')

cumulative_count = 0
for d in chunks(rel_records):
    res = gds.run_cypher('''
        UNWIND $data AS d
        MATCH(p:Provider {providerId: d.providerId})
        MATCH(t:Beneficiary {beneId: d.beneID})
        MERGE (p)-[r:HAS_CLAIMS_WITH_BENE]->(t)
        SET r.claimCount = d.claimID,
            r.inscClaimReimbursedSum = d.inscClaimAmtReimbursed,
            r.inpatientCount = d.inpatient
        RETURN count(r) AS relLoadedCount
    ''', params={'data': d})
    cumulative_count += res.relLoadedCount[0]
    print(f'Loaded {cumulative_count:,} of {total:,} relationships...')
print('Complete')

======  loading HAS_CLAIMS_WITH_BENE relationships  ======
staged 363,300 records...
Loaded 50,000 of 363,300 relationships...
Loaded 100,000 of 363,300 relationships...
Loaded 150,000 of 363,300 relationships...
Loaded 200,000 of 363,300 relationships...
Loaded 250,000 of 363,300 relationships...
Loaded 300,000 of 363,300 relationships...
Loaded 350,000 of 363,300 relationships...
Loaded 363,300 of 363,300 relationships...
Complete


We are going to repeat the above aggregation pattern for the opther relationships.  Lets create a function for it to make that easier. 

In [33]:
def load_claim_rels(gds: GraphDataScience, rel_type: str, target_node_label: str, clm_df: pd.DataFrame,
                    target_key_map: Tuple):
    print(f'======  loading {rel_type} relationships  ======')
    tmp_df = clm_df[['providerId', target_key_map[0], 'claimID', 'inscClaimAmtReimbursed', 'inpatient']] \
        .groupby(['providerId', target_key_map[0]]).agg({
            'claimID': 'count',
            'inscClaimAmtReimbursed': sum,
            'inpatient': sum
        }).reset_index()
    rel_records = tmp_df.to_dict('records')
    total = len(rel_records)
    print(f'staging {total:,} records')

    cumulative_count = 0
    for d in chunks(rel_records):
        res = gds.run_cypher(f'''
            UNWIND $data AS d
            MATCH(p:Provider {{providerId: d.providerId}})
            MATCH(t:{target_node_label} {{{target_key_map[1]}: d.{target_key_map[0]}}})
            MERGE (p)-[r:{rel_type}]->(t)
            SET r.claimCount = d.claimID,
                r.inscClaimReimbursedSum = d.inscClaimAmtReimbursed,
                r.inpatientCount = d.inpatient
            RETURN count(r) AS relLoadedCount
        ''', params={'data': d})
        cumulative_count += res.iloc[0, 0]
        print(f'Loaded {cumulative_count:,} of {total:,} relationships')

In [34]:
# Group Diagnosis Code Relationships
load_claim_rels(gds, 'HAS_CLAIMS_WITH_GROUP_CODE', 'DiagnosisGroupCode', claim_df, ('diagnosisGroupCode', 'code'))

======  loading HAS_CLAIMS_WITH_GROUP_CODE relationships  ======
staging 37,553 records
Loaded 37,553 of 37,553 relationships


In [35]:
# Diagnosis Code Relationships
diagnosis_clm_code_dfs = []
for col in claim_df.columns:
    if 'diagnosiscode' in col.lower():
        tmp_clm_df = claim_df[['providerId', 'claimID', 'inscClaimAmtReimbursed', 'inpatient', col]]\
            .dropna(subset=[col]).rename(columns={col: 'code'})
        diagnosis_clm_code_dfs.append(tmp_clm_df)
diagnosis_clm_code_df = pd.concat(diagnosis_clm_code_dfs)
load_claim_rels(gds, 'HAS_CLAIMS_WITH_CODE', 'DiagnosisCode', diagnosis_clm_code_df, ('code', 'code'))

======  loading HAS_CLAIMS_WITH_CODE relationships  ======
staging 1,001,352 records
Loaded 50,000 of 1,001,352 relationships
Loaded 100,000 of 1,001,352 relationships
Loaded 150,000 of 1,001,352 relationships
Loaded 200,000 of 1,001,352 relationships
Loaded 250,000 of 1,001,352 relationships
Loaded 300,000 of 1,001,352 relationships
Loaded 350,000 of 1,001,352 relationships
Loaded 400,000 of 1,001,352 relationships
Loaded 450,000 of 1,001,352 relationships
Loaded 500,000 of 1,001,352 relationships
Loaded 550,000 of 1,001,352 relationships
Loaded 600,000 of 1,001,352 relationships
Loaded 650,000 of 1,001,352 relationships
Loaded 700,000 of 1,001,352 relationships
Loaded 750,000 of 1,001,352 relationships
Loaded 800,000 of 1,001,352 relationships
Loaded 850,000 of 1,001,352 relationships
Loaded 900,000 of 1,001,352 relationships
Loaded 950,000 of 1,001,352 relationships
Loaded 1,000,000 of 1,001,352 relationships
Loaded 1,001,352 of 1,001,352 relationships
